In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#hyper parameters
sequence_length = 28
input_size = 28
hidden_size = 128
layer_nums = 2
output_size = 10
batch_size = 100
epoch_num = 2
learning_rate = 0.01

In [4]:
#data download
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [17]:
#RNN (many-to-one)
class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,layer_nums,output_size):
        super(RNN,self).__init__()
        self.hidden_size = hidden_size
        self.layer_nums = layer_nums
        self.lstm = nn.LSTM(input_size,hidden_size,layer_nums,batch_first = True)
        self.fc = nn.Linear(hidden_size,output_size)
        
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.layer_nums, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.layer_nums, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out
        
"""class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, layer_nums, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.layer_nums = layer_nums
        self.lstm = nn.LSTM(input_size, hidden_size, layer_nums, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.layer_nums, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.layer_nums, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out"""

'class RNN(nn.Module):\n    def __init__(self, input_size, hidden_size, layer_nums, output_size):\n        super(RNN, self).__init__()\n        self.hidden_size = hidden_size\n        self.layer_nums = layer_nums\n        self.lstm = nn.LSTM(input_size, hidden_size, layer_nums, batch_first=True)\n        self.fc = nn.Linear(hidden_size, output_size)\n\n    def forward(self, x):\n        # Set initial hidden and cell states \n        h0 = torch.zeros(self.layer_nums, x.size(0), self.hidden_size).to(device) \n        c0 = torch.zeros(self.layer_nums, x.size(0), self.hidden_size).to(device)\n\n        # Forward propagate LSTM\n        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)\n\n        # Decode the hidden state of the last time step\n        out = self.fc(out[:, -1, :])\n        return out'

In [18]:
model = RNN(input_size,hidden_size,layer_nums,output_size).to(device)

In [19]:
#criterion and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [20]:
#train the model
total_step = len(train_loader)
for epoch in range(epoch_num):
    for i,(images,labels) in enumerate(train_loader):
        images = images.reshape(-1,sequence_length,input_size).to(device)
        labels = labels.to(device)
        
        #forward
        output = model(images)
        loss = criterion(output,labels)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epoch_num, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 2.1607
Epoch [1/2], Step [200/600], Loss: 0.8088
Epoch [1/2], Step [300/600], Loss: 0.2238
Epoch [1/2], Step [400/600], Loss: 0.1932
Epoch [1/2], Step [500/600], Loss: 0.2463
Epoch [1/2], Step [600/600], Loss: 0.2533
Epoch [2/2], Step [100/600], Loss: 0.1150
Epoch [2/2], Step [200/600], Loss: 0.1377
Epoch [2/2], Step [300/600], Loss: 0.0347
Epoch [2/2], Step [400/600], Loss: 0.1056
Epoch [2/2], Step [500/600], Loss: 0.0309
Epoch [2/2], Step [600/600], Loss: 0.0930


In [22]:
model.eval()
#model test
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'RNN.ckpt')

Test Accuracy of the model on the 10000 test images: 97.89 %
